take v1.0 and just add new data to train

In [1]:
from os.path import join
from os import makedirs
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from coreml.utils.io import save_yml, read_yml

In [2]:
data_root = '/data/siim-isic-melanoma/processed/'

In [5]:
train_df = pd.read_csv(join(data_root, 'train_big.csv'))
train_small_df = pd.read_csv(join(data_root, 'train.csv'))

In [7]:
base_version = 'v1.0-256x256.yml'
target_version = 'v2.1-256x256.yml'

In [8]:
version_folder = join(data_root, 'versions')

In [9]:
base_data_config = read_yml(join(version_folder, base_version))

In [21]:
base_data_config['train']['label'][:3]

[{'classification': 0}, {'classification': 0}, {'classification': 0}]

In [12]:
old_train_files = set(train_small_df['image_name'].values)

In [13]:
new_train_indices = [index for index, file in enumerate(train_df['image_name'].values) if file not in old_train_files]

In [14]:
len(new_train_indices)

25911

In [23]:
new_train_files[:3]

['ISIC_0000000', 'ISIC_0000001', 'ISIC_0000002']

In [26]:
new_train_files = train_df.loc[new_train_indices]['image_name'].values.tolist()
new_train_labels = train_df.loc[new_train_indices]['target'].values.tolist()

In [27]:
new_train_labels = [{'classification': label} for label in new_train_labels]

In [28]:
new_train_files = [join(data_root, 'images-256x256', f'{file}.jpg') for file in new_train_files]

In [30]:
new_train_files[:4]

['/data/siim-isic-melanoma/processed/images-256x256/ISIC_0000000.jpg',
 '/data/siim-isic-melanoma/processed/images-256x256/ISIC_0000001.jpg',
 '/data/siim-isic-melanoma/processed/images-256x256/ISIC_0000002.jpg',
 '/data/siim-isic-melanoma/processed/images-256x256/ISIC_0000003.jpg']

In [33]:
base_data_config['train']['file'] += new_train_files

In [34]:
base_data_config['train']['label'] += new_train_labels

In [36]:
len(base_data_config['train']['label']), len(base_data_config['train']['file'])

(52288, 52288)

In [37]:
version_save_path = join(version_folder, target_version)

In [38]:
version_save_path

'/data/siim-isic-melanoma/processed/versions/v2.1-256x256.yml'

In [40]:
save_yml(version_save_path, base_data_config)